In [7]:
# %%
# importation des bibliothèques
import requests
import pandas as pd
import sqlite3, sqlalchemy
from tqdm.notebook import trange, tqdm
from sqlalchemy import Table, Column, Integer, String, ForeignKey, MetaData, create_engine, text, inspect, Float


# liste des urls pour chaque fichier à telecharger
urls = ['https://datasets.imdbws.com/name.basics.tsv.gz',
       'https://datasets.imdbws.com/title.basics.tsv.gz', 
       'https://datasets.imdbws.com/title.episode.tsv.gz', 
       'https://datasets.imdbws.com/title.principals.tsv.gz', 
       'https://datasets.imdbws.com/title.ratings.tsv.gz']

# # telechargement des fichiers
# for url in urls:

#     target_path = str(url[28:])
#     response = requests.get(url, stream=True)  
   
#     if response.status_code == 200:
#         with open(target_path, 'wb') as f:
#             f.write(response.raw.read())
            
#     for i in trange(1, desc='Statut'):
#         print('Fichier téléchargé :', target_path)

# %%
# création de la base de données crew.db
engine = create_engine('sqlite:///crew.db', echo=False)
meta = MetaData()

import csv

def read_tsv(path):
    with open(path, newline="", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter="\t")
        headers = next(reader)

        for row in reader:
            yield {
                h: (None if v == r"\N" else v)
                for h, v in zip(headers, row)
            }




import gzip
import csv

# with gzip.open("title.principals.tsv.gz", "rt", encoding="utf-8") as f:
#     reader = csv.DictReader(f, delimiter="\t")

#     for i, row in enumerate(reader):
#         print(row)
#         if i == 4:
            # break


#inspect data in file

with gzip.open("data/raw/name.basics.tsv.gz", "rt", encoding="utf-8") as f:
    reader = csv.DictReader(f, delimiter="\t")

    for i, row in enumerate(reader):
        print(row)
        if i == 4: 
            break



{'nconst': 'nm0000001', 'primaryName': 'Fred Astaire', 'birthYear': '1899', 'deathYear': '1987', 'primaryProfession': 'actor,miscellaneous,producer', 'knownForTitles': 'tt0072308,tt0050419,tt0027125,tt0025164'}
{'nconst': 'nm0000002', 'primaryName': 'Lauren Bacall', 'birthYear': '1924', 'deathYear': '2014', 'primaryProfession': 'actress,miscellaneous,soundtrack', 'knownForTitles': 'tt0037382,tt0075213,tt0038355,tt0117057'}
{'nconst': 'nm0000003', 'primaryName': 'Brigitte Bardot', 'birthYear': '1934', 'deathYear': '2025', 'primaryProfession': 'actress,music_department,producer', 'knownForTitles': 'tt0057345,tt0049189,tt0056404,tt0054452'}
{'nconst': 'nm0000004', 'primaryName': 'John Belushi', 'birthYear': '1949', 'deathYear': '1982', 'primaryProfession': 'actor,writer,music_department', 'knownForTitles': 'tt0072562,tt0077975,tt0080455,tt0078723'}
{'nconst': 'nm0000005', 'primaryName': 'Ingmar Bergman', 'birthYear': '1918', 'deathYear': '2007', 'primaryProfession': 'writer,director,actor

In [ ]:
# %%
################################################## title.principals.tsv.gz ##################################################
# chargement des données
df_title_principals = pd.read_csv('data/raw/title.principals.tsv.gz', compression='gzip', header=0, sep='\t')

# suppresion d'une colonne
df_title_principals.pop('ordering')
# nettoyage des données

# transformation des données du dataframe en liste de tuple
values = df_title_principals.to_records(index=False).tolist()

# création de la table ratings
crew = Table(
    'crew', meta, 
    Column('title_id', String, primary_key=False), 
    Column('person_id', String), 
    Column('category', String), 
    Column('job', String), 
    Column('characters', String) )
meta.create_all(engine)

# insertions des valeurs dans la table ratings
# on crée la connection
with engine.connect() as connection:
    # début de la transaction
    with connection.begin() as transaction:
        # on tente d'éxécuter une transaction
        try:
            # On indique le format d'un tuple de cette table
            markers = ','.join('?' * len(values[0])) 
            
            # On utilise le langage SQL en format texte où markers est le format d'un tuple
            ins = 'INSERT OR REPLACE INTO {tablename} VALUES ({markers})'
            
            # On précise ce format particulier grâce à la fonction membre format
            ins = ins.format(tablename=crew.name, markers=markers)
           
            # Enfin on peut utiliser les tuples créés en éxécutant la commande SQL
            connection.execute(ins, values)
        # si la transaction échoue
        except:
            transaction.rollback()
            raise
        # si la transaction réussit
        else:
            transaction.commit()
            
# drop des data ratings
df_title_principals = []
values = []

In [11]:
engineIMDB = create_engine('sqlite:///crew.db')
connIMDB = engineIMDB.connect()

result = connIMDB.execute("SELECT title_id,person_id,job,category,characters" 
                          " FROM crew" 
                          " WHERE category='actor'" 
                          " LIMIT 10")
result.fetchall()

[('tt0000005', 'nm0443482', '\\N', 'actor', '["Blacksmith"]'),
 ('tt0000005', 'nm0653042', '\\N', 'actor', '["Assistant"]'),
 ('tt0000007', 'nm0179163', '\\N', 'actor', '\\N'),
 ('tt0000007', 'nm0183947', '\\N', 'actor', '\\N'),
 ('tt0000008', 'nm0653028', '\\N', 'actor', '["Sneezing Man"]'),
 ('tt0000009', 'nm0183823', '\\N', 'actor', '["Mr. Hamilton"]'),
 ('tt0000009', 'nm1309758', '\\N', 'actor', '["Chauncey Depew - the Director of the New York Central Railroad"]'),
 ('tt0000011', 'nm3692297', '\\N', 'actor', '["Acrobats"]'),
 ('tt0000014', 'nm0166380', '\\N', 'actor', '["The Gardener"]'),
 ('tt0000014', 'nm0244989', '\\N', 'actor', '["The Boy"]')]